In [1]:
# ###############################################################################
##    This code generates tables 2,3,4,5 of https://arxiv.org/abs/2208.14397   ##
##    Authors: Mohit Panwar and Prabhakar Tiwari                               ##
##    Date: August 2022                                                        ##
#################################################################################
import numpy as np
import matplotlib.pyplot as plt 
import fitsio 
import healpy as hp
import sys
import scipy
import scipy.optimize as opt
import random
from numpy.linalg import norm
from numpy import (array, dot, arccos, clip)
import pandas as pd
from scipy import linalg
from scipy.stats import norm
from pynverse import inversefunc
d2r=np.pi/180.
NSIDE=64
NPIX=hp.nside2npix(NSIDE)

In [2]:
def DipoleChiSquare(A, n, values, sigma, sigma_m=None):
    if sigma_m is None:
        return np.sum(((values - (A[0] + A[1]*n[:,0] + A[2]*n[:,1] + A[3]*n[:,2]))/sigma)**2)  
    func=0
    for r,v,s_p,s_m in zip(n,values,sigma,sigma_m):
        model=A[0]+ A[1]*r[0] + A[2]*r[1] + A[3]*r[2]
        if model>v:
            func+=((v-model)/s_p)**2
        else:
            func+=((v-model)/s_m)**2   
    return func

def QuadChiSqaure(A, n, values, sigma,sigma_m=None):
    p = A[1]*n[:,0] + A[2]*n[:,1] + A[3]*n[:,2]
    dxy = A[4]*n[:,0]*n[:,1]
    dyz = A[5]*n[:,1]*n[:,2]
    dz2 = A[6]*(- n[:,0]*n[:,0] - n[:,1]*n[:,1] + 2*n[:,2]*n[:,2])
    dxz = A[7]*n[:,0]*n[:,2]
    dx2_y2 = A[8]*(n[:,0]*n[:,0] - n[:,1]*n[:,1])
    y = A[0] + p + dxy + dyz + dz2 + dxz + dx2_y2
    if sigma_m is None:
        return np.sum(((values - y)/sigma)**2)
    func = 0
    for vi, yi, s_p, s_m in zip(values, y,sigma,sigma_m):
        if yi>vi:
            func+=((vi-yi)/s_p)**2
        else:
            func+=((vi-yi)/s_m)**2
    return func

def vec2dir(vec):
    """
    Converts a Cartesian vector with unit length into longitude and latitude in degrees
    """
    UnitVector = vec/linalg.norm(vec)
    x, y, z = UnitVector
    theta = np.arccos(z)
    phi = np.arctan2(y,x)   
    lat, lon = 90.0 - np.rad2deg(theta), np.rad2deg(phi)

    if lon<0:
        return 360+lon, lat
    else:
        return lon, lat

    
def DipoleAmplitudeDirection(parameters):
    """Returns Dipole Amplitude(Dipole/Monopole) and Direction"""
    Amplitude = linalg.norm(parameters[1:4]/parameters[0])
    Direction = vec2dir(parameters[1:4]/parameters[0])
    return Direction, Amplitude

def ChiSqErrParams(chi2_fun, values):
    """Return errors in parameters"""
    N = len(values)
    A_contour = np.zeros(N)
    chi2_tot = chi2_fun(values)

    for i in range(N):
        def cost_error(x):
            params = np.copy(values)
            params[i] = x
            return chi2_fun(params)

        A_contour[i] = inversefunc(cost_error, y_values = chi2_tot + 1)

    return A_contour - values

def ErrorAmplitudeDipole(params, e_params):
    first_term = (e_params[0]*e_params[0])/(params[0]*params[0])
    sec_nume   = np.dot(params[1:4]*params[1:4], e_params[1:4]*e_params[1:4])
    sec_deno   = (np.dot(params[1:4], params[1:4]))**2 
    
    relative_error = first_term  + (sec_nume/sec_deno)
    amp_di = linalg.norm((params[1:4])/params[0])    # Amplitude = (Dipole/Monopole)
    return amp_di*np.sqrt(relative_error)

def ErrorThetaPhi(params, errors_params):
    """Returns errors in the RA(Phi), Dec(90.0 - Theta) in [deg.]. """
    sig = abs(errors_params)
    amp, err = DipoleAmplitudeError(params[1:4], sig[1:4])  # Amplitude(Dipole compenents only) and error in amplitude
    delta_phi = np.sqrt(params[1]*params[1]*errors_params[2]**2 + params[2]*params[2]*errors_params[1]**2)/(params[1]**2 + params[2]**2)
    k1 = 1.0/(amp*np.sqrt(params[1]**2 + params[2]**2))
    k2 =  np.sqrt(amp*amp*errors_params[3]**2 + params[3]*params[3]*err**2) 
    return np.rad2deg(delta_phi), np.rad2deg(k1*k2) # e_phi, e_theta

def DipoleAmplitudeError(params, e_params):
    """Returns error in dipole amplitude using error propagation formula"""
    Amplitude = np.sqrt(np.dot(params,params))
    Error  = (1.0/Amplitude)*np.sqrt(np.dot(params*params,e_params*e_params))
    return Amplitude, Error

def GetChiSquaresDip(data, vecDirec, sigma, x0,sigma_m=None, methods='BFGS',printresults=True):
    DipoleParameters = opt.minimize(lambda x: DipoleChiSquare(x, vecDirec, data, sigma,sigma_m=sigma_m), x0, method=methods)
    dirr, Ampl = DipoleAmplitudeDirection(DipoleParameters.x)
    if printresults:
        print("Chi-Squares Minimization")
        print('sum of least-squares',DipoleParameters.fun)  # chi-square
        print('fitting parameters',  DipoleParameters.x)    # parameters
        print('Direction and Amplitude')
        print(dirr, Ampl)
    return dirr[0],dirr[1],Ampl,DipoleParameters.fun/(len(data)-4)

def Quadrupole(A, n):
    dxy = n[0]*n[1]
    dyz = n[1]*n[2]
    dz2 = (- n[0]*n[0] - n[1]*n[1] + 2*n[2]*n[2])
    dxz = n[0]*n[2]
    dx2_y2 = (n[0]*n[0] - n[1]*n[1])
    
    return np.dot(A, np.array([dxy, dyz, dz2, dxz, dx2_y2]))
    
def QuadrupoleDirection(quadParam, nside):
    r = np.array([Quadrupole(quadParam,hp.pix2vec(nside, pix)) for pix in range(hp.nside2npix(nside))])
    
    return hp.pix2ang(nside, r.argsort()[-2:], lonlat=True)
    
def GetChiSquaresQuad(data, vecDirec, sigma, x0,sigma_m=None, methods='BFGS',printresults=True):
    quadrupoleParams = opt.minimize(lambda x: QuadChiSqaure(x, vecDirec, data, sigma,sigma_m=sigma_m), x0, method=methods)
    dirr, Ampl = DipoleAmplitudeDirection(quadrupoleParams.x)
    Quad_params_error = ChiSqErrParams(lambda x: QuadChiSqaure(x, vecDirec, data, sigma), quadrupoleParams.x)
    mag_err = ErrorAmplitudeDipole(quadrupoleParams.x, Quad_params_error)
    ra_err, dec_err = ErrorThetaPhi(quadrupoleParams.x, Quad_params_error)
    if printresults:
        print('Chi-Squares Minimization')
        print("Chi-Squares", quadrupoleParams.fun)
        print('Reduced Chi-Sqaures =',quadrupoleParams.fun/(len(data)-10))
        print("Quadrupole fitting Parameters")
        print(quadrupoleParams.x)
        print(dirr, Ampl)
        #print(quaDir)
    return dirr[0],dirr[1],Ampl,quadrupoleParams.fun/(len(data)-10),mag_err,ra_err, dec_err

def get_fsky(m):
    count=0  
    for pix in range(NPIX):
        if m[pix]>0:
            count+=1
    #print("pixels with data:", count)
    #print("f_sky:", count/NPIX)
    return count/NPIX

def mask_neighbour(m):
    NPIX=len(m)
    NSIDE=hp.npix2nside(NPIX)
    mask=np.ones(NPIX)
    for pix in range(NPIX):
        if m[pix]<=5:
            mask[pix]=0
            if m[pix]<=0:
                lon,lat=hp.pix2ang(NSIDE,pix,nest=False, lonlat=True)
                npix_a=hp.get_all_neighbours(NSIDE,lon,lat,nest=False, lonlat=True)
                for p in npix_a:
                    mask[p]=0              
    return mask

def apply_mask(m,mask):
    for pix in range(NPIX):
          if mask[pix]==0:
                m[pix]=hp.UNSEEN
    return m


def get_asymmetric_errors(flux_data,nn,draw=True):
    data=[]
    fsum=0.0
    for i, f in enumerate(flux_data):
        fsum+=f
        if i!=0 and i % int(nn)==0:
            data.append(fsum/nn)
            fsum=0.0   

    values,bins=np.histogram(data, bins=np.linspace(0,2.0,400), 
                                     density=True)
    freq=0
    median=-1
    i=0
    for fq,b_l,b_r in zip(values,bins,bins[1:]):
        #print(fq,0.5*(b_l+b_r))
        if fq>freq:
            freq=fq
            median=0.5*(b_l+b_r)
        i+=1
    #print("median, freq:=",median,freq,np.median(data))


    ## get asymmetric 

    mu=median
    sort_data=sorted(data)
    nn=len(sort_data)
    count_34p1=int(nn*0.341)
    for i,s in enumerate(sort_data):
        if s>mu:
            index=i
            break
    #print(nn,count_34p1,index)
    if index-count_34p1>0:
        sigma_m=mu-sort_data[index-count_34p1]
    else:
        sigma_m=mu-sort_data[0]
    if index+count_34p1>nn-1:
        sigma_p=sort_data[nn-1]-mu
    else:
        sigma_p=sort_data[index+count_34p1]-mu

    if draw:
        plt.hist(data, bins=np.linspace(0,2.0,400),histtype='step', 
                                     density=True, alpha=0.6, color='b')
        print(mu,sigma_m,sigma_p)
        plt.axvline(x = median,c='r',ls='-')
        plt.xlim(0.1,0.4)
        plt.axvline(x = median-sigma_m,c='r',ls='--')
        plt.axvline(x = median+sigma_p,c='r',ls='--')
        plt.xlabel(r"$\bar{S}$")
        plt.show()
    return sigma_m,sigma_p

In [3]:
def demo():
    file = 'alpha_avg_map_lonlat_fcut.dat'
    DFile = pd.read_table(file, sep="\s+",skiprows=1,header=None)
    l_arr, b_arr = np.array(DFile)[:,0], np.array(DFile)[:,1]
    dat_arr, sigma_arr = np.array(DFile)[:,2], np.array(DFile)[:,3]
    vec_arr = hp.ang2vec(l_arr, b_arr, lonlat=True)
    x0 = [1.0, 0.1, 0.1, 1.0]
    GetChiSquaresDip(dat_arr, vec_arr, sigma_arr, x0,sigma_m=sigma_arr)
    GetChiSquaresDip(dat_arr, vec_arr, sigma_arr, x0)
    x0 = [10.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
    GetChiSquaresQuad(dat_arr, vec_arr, sigma_arr, x0)
    GetChiSquaresQuad(dat_arr, vec_arr, sigma_arr, x0,sigma_m=sigma_arr)

In [4]:
#demo()
### read data from fits file 
filein="catwise_agns_masked_final_w1lt16p5_alpha.fits"
column = ['ra','dec','l','b','w1','w2','k','alpha_W1','nu_W1_iso']
data,h= fitsio.read(filein,columns=column,header=True)
#print(h)
RA_raw,DEC_raw,l_raw,b_raw,w1_raw,w2_raw=data['ra'],data['dec'],data['l'],data['b'], data['w1'],data['w2']
k_raw,alpha_raw,nu_raw=data['k'],data['alpha_W1'],data['nu_W1_iso']
ra_all,dec_all,flux_all,alpha_all=[],[],[],[]
count=0
for ra,dec,l,b,w1,w2,k,alpha,nu in zip(RA_raw,DEC_raw,l_raw,b_raw,w1_raw,w2_raw,k_raw,alpha_raw,nu_raw):
    if w1-w2<0.8:
        print(ra, dec,w1-w2)
        count+=1
    if 9<w1<16.4:

        #ra_data.append(ra)
        #dec_data.append(dec)
        flux=k*np.power(nu,alpha)*1.0E26 # cgs to mJy conversion 
        ra_all.append(l)  # Use Galactic coordinate 
        dec_all.append(b)
        alpha_all.append(-alpha)
        flux_all.append(flux)
print(max(flux_all))

73.24003656157868


In [5]:
def gen_table2():
    header="# S_cut(<mJy)  Source f_sky     |Dipole|     l     b"
    print(header)
    fout=open("table2.dat","w")
    fout.write("%s\n"%header)
    for f_cut in [np.inf, 10.0, 0.7,0.3, 0.2,0.1]:
        ra_data,dec_data,flux_data,alpha_data=[],[],[],[]
        for l,b,alpha,f in zip(ra_all,dec_all,alpha_all,flux_all):
            if f<f_cut:
                ra_data.append(l)
                dec_data.append(b)
                alpha_data.append(alpha)

        m_nn=np.zeros(NPIX)
        m_alp=np.zeros(NPIX)
        for ra,dec,alpha in zip(ra_data,dec_data,alpha_data):
            pix=hp.ang2pix(NSIDE, ra,dec,lonlat=True,nest=False)
            m_nn[pix]=m_nn[pix]+1
            m_alp[pix]=m_alp[pix]+alpha 

        m_alpha=np.zeros(NPIX)
        for pix in range(NPIX):
            if m_nn[pix]>0:
                m_alpha[pix]=m_alp[pix]/m_nn[pix]

        try:
            mask
        except NameError:
            pass
        else:
            del mask 

        mask=mask_neighbour(m_nn)

        m_nn=apply_mask(m_nn,mask)
        m_alpha=apply_mask(m_alpha,mask)
        fsky=get_fsky(m_nn)
        data_arr,l_arr,b_arr, sigma_arr_p,sigma_arr_m=[],[],[],[],[]
        nn_arr=[]
        nn=0

        for pix in range(NPIX):
            if mask[pix]>0:
                nn+=m_nn[pix]
                l,b=hp.pix2ang(NSIDE,pix,nest=False,lonlat=True)
                data_arr.append(m_alpha[pix])
                l_arr.append(l)
                b_arr.append(b)
                nn_arr.append(m_nn[pix])

        min_nn,max_nn=int(min(nn_arr)),int(max(nn_arr))
        sigma_pix_m,sigma_pix_p=[],[]
        for n in range(min_nn, max_nn+1):
            sig_m,sig_p=get_asymmetric_errors(alpha_data,n,draw=False)
            sigma_pix_m.append(sig_m)
            sigma_pix_p.append(sig_p)
        #print("Sigma array done!")
        for n in nn_arr:
            index=int(n-min_nn)
            sig_m,sig_p=sigma_pix_m[index],sigma_pix_p[index]
            sigma_arr_m.append(sig_m)
            sigma_arr_p.append(sig_p)

        try:
            full_nn
        except NameError:
            full_nn=nn
        else:
            pass

        vec_arr=hp.ang2vec(l_arr, b_arr, lonlat=True)    

        if len(data_arr)>10:
            x0 = [10.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
            lq,bq,ampq,chi2dofq,e_d,e_l,e_b=GetChiSquaresQuad(data_arr, vec_arr, sigma_arr_p, x0,sigma_m=sigma_arr_m,printresults=False)

            string="%5.1f %7d %0.4f  %.4f+-%0.5f  %3.0f+-%2.0f  %3.0f+-%2.0f"%(float(f_cut),int(nn),fsky,ampq,e_d,lq,e_l,bq,e_b)
            fout.write("%s\n"%string)
            print(string)
        else:
            print("Not enough data points to fit")
    fout.close()
    del full_nn
    print("Done!")

In [6]:
def gen_table3():
    header="# |b|>  Sources  f_sky    |Dipole|        l        b"
    print(header)
    fout=open("table3.dat","w")
    fout.write("%s\n"%header)
    for g_cut in [30,35,40,45,50]:
        ra_data,dec_data,flux_data,alpha_data=[],[],[],[]
        for l,b,alpha in zip(ra_all,dec_all,alpha_all):
            if abs(b)>g_cut:
                ra_data.append(l)
                dec_data.append(b)
                alpha_data.append(alpha)


        NSIDE=64
        NPIX=hp.nside2npix(NSIDE)
        m_nn=np.zeros(NPIX)
        m_alp=np.zeros(NPIX)
        for ra,dec,alpha in zip(ra_data,dec_data,alpha_data):
            pix=hp.ang2pix(NSIDE, ra,dec,lonlat=True,nest=False)
            m_nn[pix]=m_nn[pix]+1
            m_alp[pix]=m_alp[pix]+alpha 


        m_alpha=np.zeros(NPIX)
        for pix in range(NPIX):
            if m_nn[pix]>0:
                m_alpha[pix]=m_alp[pix]/m_nn[pix]

        try:
            mask
        except NameError:
            pass
        else:
            del mask 

        mask=mask_neighbour(m_nn)

        m_nn=apply_mask(m_nn,mask)
        m_alpha=apply_mask(m_alpha,mask)
        fsky=get_fsky(m_nn)

        data_arr,l_arr,b_arr, sigma_arr_p,sigma_arr_m=[],[],[],[],[]
        nn_arr=[]
        nn=0

        for pix in range(NPIX):
            if mask[pix]>0:
                nn+=m_nn[pix]
                l,b=hp.pix2ang(NSIDE,pix,nest=False,lonlat=True)
                data_arr.append(m_alpha[pix])
                l_arr.append(l)
                b_arr.append(b)
                nn_arr.append(m_nn[pix])

        min_nn,max_nn=int(min(nn_arr)),int(max(nn_arr))
        sigma_pix_m,sigma_pix_p=[],[]
        for n in range(min_nn, max_nn+1):
            sig_m,sig_p=get_asymmetric_errors(alpha_data,n,draw=False)
            sigma_pix_m.append(sig_m)
            sigma_pix_p.append(sig_p)

        for n in nn_arr:
            index=int(n-min_nn)
            sig_m,sig_p=sigma_pix_m[index],sigma_pix_p[index]
            sigma_arr_m.append(sig_m)#sigma_m/sqrtn)
            sigma_arr_p.append(sig_p)#sigma_p/sqrtn)

        try:
            full_nn
        except NameError:
            full_nn=nn
        else:
            pass

        vec_arr=hp.ang2vec(l_arr, b_arr, lonlat=True)    

        if len(data_arr)>10:
            x0 = [10.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
            lq,bq,ampq,chi2dofq,e_d,e_l,e_b=GetChiSquaresQuad(data_arr, vec_arr, sigma_arr_p, x0,sigma_m=sigma_arr_m,printresults=False)
            string="%2.0f %7d %0.4f  %.4f+-%0.5f  %3.0f+-%2.0f  %3.0f+-%2.0f"%(float(g_cut),int(nn),fsky,ampq,e_d,lq,e_l,bq,e_b)
            fout.write("%s\n"%string)
            print(string)
        else:
            print("Not enough data points to fit")
    fout.close()

    del full_nn
    print("Done!")

In [7]:
#generate table 4
def gen_table4():
    header="# S_cut(<mJy)  Source f_sky     |Dipole|     l     b"
    print(header)
    fout=open("table4.dat","w")
    fout.write("%s\n"%header)
    for f_cut in [np.inf,10.0, 1.0, 0.7,0.3, 0.2,0.1]:
        ra_data,dec_data,flux_data=[],[],[]
        for l,b,f in zip(ra_all,dec_all,flux_all):
            if f<f_cut:
                ra_data.append(l)
                dec_data.append(b)
                flux_data.append(f)

        NSIDE=64
        NPIX=hp.nside2npix(NSIDE)
        m_nn=np.zeros(NPIX)
        m_bb=np.zeros(NPIX)
        for f,ra,dec in zip(flux_data,ra_data,dec_data):
            pix=hp.ang2pix(NSIDE, ra,dec,lonlat=True,nest=False)
            m_nn[pix]=m_nn[pix]+1
            m_bb[pix]=m_bb[pix]+f

        ## Get brightness_avg 
        m_bb_avg=np.zeros(NPIX)
        for pix in range(NPIX):
            if m_nn[pix]>0:
                m_bb_avg[pix]= m_bb[pix]/m_nn[pix]

        try:
            mask
        except NameError:
            pass
        else:
            del mask 

        mask=mask_neighbour(m_nn)

        m_nn=apply_mask(m_nn,mask)
        m_bb=apply_mask(m_bb,mask)
        m_bb_avg=apply_mask(m_bb_avg,mask)
        fsky=get_fsky(m_nn)

        data_arr,l_arr,b_arr, sigma_arr_p,sigma_arr_m=[],[],[],[],[]
        nn_arr=[]
        nn=0

        for pix in range(NPIX):
            if mask[pix]>0:
                nn+=m_nn[pix]
                l,b=hp.pix2ang(NSIDE,pix,nest=False,lonlat=True)
                data_arr.append(m_bb_avg[pix])
                l_arr.append(l)
                b_arr.append(b)
                nn_arr.append(m_nn[pix])

        min_nn,max_nn=int(min(nn_arr)),int(max(nn_arr))
        sigma_pix_m,sigma_pix_p=[],[]
        for n in range(min_nn, max_nn+1):
            sig_m,sig_p=get_asymmetric_errors(flux_data,n,draw=False)
            sigma_pix_m.append(sig_m)
            sigma_pix_p.append(sig_p)

        for n in nn_arr:
            index=int(n-min_nn)
            sig_m,sig_p=sigma_pix_m[index],sigma_pix_p[index]
            sigma_arr_m.append(sig_m)
            sigma_arr_p.append(sig_p)

        vec_arr=hp.ang2vec(l_arr, b_arr, lonlat=True) 
        try:
            full_nn
        except NameError:
            full_nn=nn
        else:
            pass


        #print(nn,len(data_arr))
        if len(data_arr)>10:
            x0 = [10.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
            lq,bq,ampq,chi2dofq,e_d,e_l,e_b=GetChiSquaresQuad(data_arr, vec_arr, sigma_arr_p, x0,sigma_m=sigma_arr_m,printresults=False)
            string="%5.1f %7d %0.4f  %.4f+-%0.5f  %3.0f+-%2.0f  %3.0f+-%2.0f"%(float(f_cut),int(nn),fsky,ampq,e_d,lq,e_l,bq,e_b)
            fout.write("%s\n"%string)
            print(string)
        else:
            print("Not enough data points to fit")
    fout.close()
    del full_nn
    print("Done!")

In [8]:
def gen_table5(): 
    header="# |b|>  Sources  f_sky    |Dipole|        l        b"

    fout=open("table5.dat","w")
    print(header)
    fout.write("%s\n"%header)
    for g_cut in [30,35,40,45,50]:
        ra_data,dec_data,flux_data,alpha_data=[],[],[],[]
        for l,b,f,alpha in zip(ra_all,dec_all,flux_all,alpha_all):
            if abs(b)>g_cut:
                ra_data.append(l)
                dec_data.append(b)
                flux_data.append(f)

        #print(len(ra_data)) 
        ## Get all maps 
        NSIDE=64
        NPIX=hp.nside2npix(NSIDE)
        m_nn=np.zeros(NPIX)
        m_bb=np.zeros(NPIX)
        for f,ra,dec in zip(flux_data,ra_data,dec_data):
            pix=hp.ang2pix(NSIDE, ra,dec,lonlat=True,nest=False)
            m_nn[pix]=m_nn[pix]+1
            m_bb[pix]=m_bb[pix]+f

        ## Get brightness_avg and alpha_avg map
        m_bb_avg=np.zeros(NPIX)
        for pix in range(NPIX):
            if m_nn[pix]>0:
                m_bb_avg[pix]= m_bb[pix]/m_nn[pix]


        try:
            mask
        except NameError:
            pass
        else:
            del mask 

        mask=mask_neighbour(m_nn)

        m_nn=apply_mask(m_nn,mask)
        m_bb_avg=apply_mask(m_bb_avg,mask)

        fsky=get_fsky(m_nn)
        data_arr,l_arr,b_arr, sigma_arr_p,sigma_arr_m=[],[],[],[],[]
        nn_arr=[]
        nn=0

        for pix in range(NPIX):
            if mask[pix]>0:
                nn+=m_nn[pix]
                l,b=hp.pix2ang(NSIDE,pix,nest=False,lonlat=True)
                data_arr.append(m_bb_avg[pix])
                l_arr.append(l)
                b_arr.append(b)
                nn_arr.append(m_nn[pix])

        min_nn,max_nn=int(min(nn_arr)),int(max(nn_arr))
        sigma_pix_m,sigma_pix_p=[],[]
        for n in range(min_nn, max_nn+1):
            sig_m,sig_p=get_asymmetric_errors(flux_data,n,draw=False)
            sigma_pix_m.append(sig_m)
            sigma_pix_p.append(sig_p)

        for n in nn_arr:
            index=int(n-min_nn)
            sig_m,sig_p=sigma_pix_m[index],sigma_pix_p[index]
            sigma_arr_m.append(sig_m)
            sigma_arr_p.append(sig_p)


        try:
            full_nn
        except NameError:
            full_nn=nn
        else:
            pass

        vec_arr=hp.ang2vec(l_arr, b_arr, lonlat=True)    
        if len(data_arr)>10:
            x0 = [10.0,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1]
            lq,bq,ampq,chi2dofq,e_d,e_l,e_b=GetChiSquaresQuad(data_arr, vec_arr, sigma_arr_p, x0,sigma_m=sigma_arr_m,printresults=False)
            string="%2.0f %7d %0.4f  %.4f+-%0.5f  %3.0f+-%2.0f  %3.0f+-%2.0f"%(float(g_cut),int(nn),fsky,ampq,e_d,lq,e_l,bq,e_b)
            fout.write("%s\n"%string)
            print(string)
        else:
            print("Not enough data points to fit")
    fout.close()
    del full_nn
    print("Done!")

In [ ]:
gen_table2()
gen_table3()
gen_table4()
gen_table5()

# S_cut(<mJy)  Source f_sky     |Dipole|     l     b
  inf 1307511 0.4724  0.0065+-0.00113  171+- 6    7+- 6
 10.0 1307023 0.4724  0.0067+-0.00099  172+- 5    9+- 4
  0.7 1263875 0.4722  0.0067+-0.00148  167+- 9    8+- 5
  0.3 1120488 0.4721  0.0065+-0.00090  168+-12   10+- 6
  0.2  953664 0.4718  0.0064+-0.00071  172+-11   10+- 4
  0.1  302607 0.4647  0.0070+-0.00372  217+-43   22+-20
Done!
# |b|>  Sources  f_sky    |Dipole|        l        b
30 1307511 0.4724  0.0065+-0.00113  171+- 6    7+- 6
35 1117078 0.4030  0.0074+-0.00147  161+- 9    5+- 6
40  950186 0.3426  0.0055+-0.00099  172+-14    4+- 5
45  767093 0.2765  0.0054+-0.00135  150+-14    7+-10
50  619814 0.2235  0.0036+-0.00184  147+-26   14+-15
Done!
# S_cut(<mJy)  Source f_sky     |Dipole|     l     b
  inf 1307511 0.4724  0.0121+-0.00303  352+-60   10+- 8
 10.0 1307023 0.4724  0.0081+-0.00172  356+-16   11+-17
  1.0 1284744 0.4724  0.0029+-0.00128  354+-78   38+-27
  0.7 1263875 0.4722  0.0023+-0.00089  306+-89   65+-59
  0.